In [69]:
import numpy as np
import networkx as nx
import random
import copy
from sklearn.preprocessing import normalize

In [51]:
max_n = 50
min_n = 30
cur_n = np.random.randint(max_n - min_n + 1) + min_n

In [52]:
g = nx.barabasi_albert_graph(n=cur_n, m=4)



In [53]:
def set_edge_weight(g):
    a=nx.adjacency_matrix(g).todense()
    a=np.array(a+np.eye(cur_n))
    b=np.random.rand(cur_n,cur_n)
    c=a*b
    edge_weight = normalize(c, axis=0, norm='l1')
    return edge_weight


In [56]:
seed=random.sample(g.nodes(),5)

In [58]:
if type(g) == nx.MultiGraph or type(g) == nx.MultiDiGraph:
      raise Exception( \
          "linear_threshold() is not defined for graphs with multiedges.")

for s in seed:
    if s not in g.nodes():
      raise Exception("seed", s, "is not in graph")



In [60]:
# change to directed graph
if not g.is_directed():
    DG = g.to_directed()
else:
    DG = copy.deepcopy(g)        # copy.deepcopy 深拷贝 拷贝对象及其子对象

In [61]:
# init thresholds
for n in DG.nodes():
    if 'threshold' not in DG.node[n]:
        DG.node[n]['threshold'] = random.random()
    elif DG.node[n]['threshold'] > 1:
        raise Exception("node threshold:", DG.node[n]['threshold'], \
      "cannot be larger than 1")

In [62]:
edge_weight = set_edge_weight(DG)

In [63]:
# init influence weight
in_deg = DG.in_degree()       #获取所有节点的入度
for e in DG.edges():
    if 'influence' not in DG[e[0]][e[1]]:
        DG[e[0]][e[1]]['influence'] =  edge_weight[e[0]][e[1]]   #计算边的权重
    elif DG[e[0]][e[1]]['influence'] > 1:
        raise Exception("edge influence:", DG[e[0]][e[1]]['influence'], \
      "cannot be larger than 1")

In [65]:
def _influence_sum(G, froms, to):
  influence_sum = 0.0
  for f in froms:
    influence_sum += G[f][to]['influence']
  return influence_sum

In [66]:
def _diffuse_one_round(G, A):
  activated_nodes_of_this_round = set()
  for s in A:
    nbs = G.successors(s)
    for nb in nbs:
      if nb in A:
        continue
      active_nb = list(set(G.predecessors(nb)).intersection(set(A)))
      if _influence_sum(G, active_nb, nb) >= G.node[nb]['threshold']:
        activated_nodes_of_this_round.add(nb)
  A.extend(list(activated_nodes_of_this_round))
  return A, list(activated_nodes_of_this_round)

In [67]:
def _diffuse_all(G, A):
  layer_i_nodes = [ ]
  layer_i_nodes.append([i for i in A])
  while True:
    len_old = len(A)
    A, activated_nodes_of_this_round = _diffuse_one_round(G, A)
    layer_i_nodes.append(activated_nodes_of_this_round)
    if len(A) == len_old:
      break
  return layer_i_nodes

In [71]:
# perform diffusion
A = copy.deepcopy(seed)

In [72]:
result=_diffuse_all(DG, A)



In [73]:
result

[[13, 20, 31, 23, 29],
 [9, 18, 3, 7],
 [37, 12, 47, 17, 19, 25, 30],
 [24, 41, 21],
 [22],
 []]